In [1]:
import pandas as pd

In [2]:
pv_raw = pd.read_csv('datasets/1976-2020-president.csv')
years = set(pv_raw['year'])
states = set(pv_raw['state'])

In [3]:
pv_raw = pv_raw[['year', 'state', 'party_simplified', 'candidatevotes', ]] # drop useless columns
pv_raw.groupby(['year', 'state', 'party_simplified', ]).sum() # drop multiple candidate
pv_raw = pv_raw[pv_raw['party_simplified'].isin(['DEMOCRAT', 'REPUBLICAN' ])] # exclude OTHER party
pv_raw = pv_raw.set_index(['year', 'state', 'party_simplified', ])
pv_raw.head()

candidatevotes
year state   party_simplified                
1976 ALABAMA DEMOCRAT                  659170
             REPUBLICAN                504070
     ALASKA  REPUBLICAN                 71555
             DEMOCRAT                   44058
     ARIZONA REPUBLICAN                418642

In [4]:
pv_clean = []
for y in years:
    for s in states:
        d = pv_raw.loc[y, s, 'DEMOCRAT'].iloc[0]['candidatevotes']
        r = pv_raw.loc[y, s, 'REPUBLICAN'].iloc[0]['candidatevotes']
        pv_clean.append([int(y), s, 100*d/(d+r)]) # % popular vote for each state

/tmp/ipykernel_638/983951477.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  d = pv_raw.loc[y, s, 'DEMOCRAT'].iloc[0]['candidatevotes']
/tmp/ipykernel_638/983951477.py:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  r = pv_raw.loc[y, s, 'REPUBLICAN'].iloc[0]['candidatevotes']


In [5]:
pv_clean = pd.DataFrame(data=pv_clean, columns=['year', 'state', 'vote', ])
#pv_clean = pv_clean.set_index(['year', 'state', ])
pv_clean.head()

,year,state,vote
0,1984,COLORADO,35.634230
1,1984,MINNESOTA,50.090891
2,1984,WEST VIRGINIA,44.727566
3,1984,LOUISIANA,38.580839
4,1984,ARKANSAS,38.772412


In [6]:
a = pd.read_csv('datasets/gdp-77-97.csv')
b = pd.read_csv('datasets/gdp-97-20.csv')
a.head()

,GeoFips,GeoName,1977,1978,1979,1980,1981,1982,1983,1984,...,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997
0,1000,Alabama,59478.4,63298.0,64987.6,64582.3,65771.0,64134.1,67305.0,70481.4,...,82507.5,82407.7,83766.4,86115.5,89813.9,91167.6,94803.1,98023.9,101379.3,104805.2
1,2000,Alaska,16933.3,18450.3,19360.9,22439.8,25865.6,26542.8,25822.1,26853.7,...,27843.4,28985.5,28772.4,25821.4,26150.7,26047.8,26187.9,27608.5,27271.2,27581.3
2,4000,Arizona,44588.9,49253.9,53887.8,55560.6,57106.1,55762.7,58949.6,65288.2,...,80258.3,80971.1,81606.3,81945.8,90282.0,94916.4,104104.4,112019.5,120868.5,129279.1
3,5000,Arkansas,32633.1,34765.4,35288.5,34773.1,36018.1,35055.0,36179.0,39015.1,...,43158.4,44011.8,44496.2,46497.8,49260.3,50756.0,53640.5,55836.1,58346.3,60332.9
4,6000,California,523795.5,559842.6,581985.9,599506.9,619242.6,619419.8,641584.7,693094.9,...,848272.5,880307.0,906102.9,893112.2,891630.9,888069.6,904778.2,940599.3,978299.5,1037091.2


In [7]:
gdp = [] 

# from row-column to table-list

for year in a.columns[2:]:
    for index, row in a.iterrows():
        gdp.append([int(year), row['GeoName'].upper(), row[year], ])

for year in b.columns[2:]:
    for index, row in b.iterrows():
        gdp.append([int(year), row['GeoName'].upper(), row[year], ])

In [8]:
gdp = [i for i in gdp if i[1] != 'NEW ENGLAND'] # remove new england

In [9]:
# dump
gdp = pd.DataFrame(data=gdp, columns=['year', 'state', 'gdp', ])
#gdp = gdp.set_index(['year', 'state', ])
gdp.to_csv('datasets/gdp.csv', index=False)
gdp.head()

,year,state,gdp
0,1977,ALABAMA,59478.4
1,1977,ALASKA,16933.3
2,1977,ARIZONA,44588.9
3,1977,ARKANSAS,32633.1
4,1977,CALIFORNIA,523795.5


In [12]:
df = pd.merge(pv_clean, gdp, how='inner', left_on=['year','state'], right_on=['year','state'])
df.to_csv('datasets/xxx-final-dataset.csv', index=False)
df.head()

,year,state,vote,gdp
0,1984,COLORADO,35.634230,80299.3
1,1984,MINNESOTA,50.090891,100958.2
2,1984,WEST VIRGINIA,44.727566,28766.2
3,1984,LOUISIANA,38.580839,102250.7
4,1984,ARKANSAS,38.772412,39015.1
